# Imports

In [3]:
# from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas_profiling import ProfileReport

# Tutorial on Beautiful Soup

In [5]:
page=1
URL = f"https://www.property24.com/to-rent/western-cape/9/p{page}?PropertyCategory=House%2cApartmentOrFlat"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "lxml")

In [6]:
soup


<!DOCTYPE html>
<!--[if lt IE 8]><html class=" p24_live p24_outdated"><![endif]--><!--[if IE 8]><html class=" p24_live ie8"><![endif]--><!--[if IE 9]><html class=" p24_live ie9"><![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="p24_live" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<!--<![endif]-->
<head>
<link href="/manifest.json" rel="manifest"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-regular.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-300.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-600.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-700.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/font-awesome/fonts/

In [7]:
properties = soup.find_all('div', class_='js_resultTile')
# properties[4].find('span', class_='left rentalTerm p24_rentalTerm').text.strip()

# Property24 Scraper

In [8]:
columns = ['id', 'title', 'description', 'bedrooms', 'bathrooms', 'parking_spaces', 'floor_area', 'location', 'address', 'rental_term', 'price']
df = pd.DataFrame([])
page = 0

while True:
    try:
        page += 1

        # URL = f"https://www.property24.com/apartments-to-rent/western-cape/9/p{page}"
        URL = f"https://www.property24.com/to-rent/western-cape/9/p{page}?PropertyCategory=House%2cApartmentOrFlat"
        soup = BeautifulSoup(requests.get(URL).content, "lxml")

        try:
            soup.find('div', {'class': 'panel-body text-danger text-center'}).h4.text
            break
        except:
            properties = soup.find_all('div', class_='js_resultTile')

        for property in properties:
            id = property['data-listing-number']
            
            try:
                price = property.find('span', class_='p24_price')['content'].strip()
            except:
                try:
                    price = property.find('div', class_='p24_price').text.strip().strip('R').replace(' ', '')
                except:
                    price = 'na'

            try:
                title = property.find('span', class_='p24_title').text.strip()
            except:
                try:
                    title = property.find('div', class_='p24_description').text.strip()
                except:
                    title = 'na'

            try:
                location = property.find('span', class_='p24_location').text.strip()
            except:
                try:
                    location = property.find('div', class_='p24_location').text.strip()
                except:
                    location = 'na'

            try:
                description = property.find('span', class_='p24_excerpt').text.strip()
            except:
                try:
                    description = property.find('span', class_='p24_excerpt').text.strip()
                except:
                    description = 'na'

            try:
                property.find('span', class_='left rentalTerm p24_rentalTerm').text.strip()
                rental_term = 'daily'
            except:
                rental_term = 'monthly'

            try:
                address = property.find('span', class_='p24_address').text.strip()
            except:
                address = 'na'

            try:
                bedrooms = property.find('span', {"class": "p24_featureDetails", 'title':'Bedrooms'}).text.strip()
            except:
                bedrooms = 'na'

            try:
                bathrooms = property.find('span', {"class": "p24_featureDetails", 'title':'Bathrooms'}).text.strip()
            except:
                bathrooms = 'na'

            try:
                parking_spaces = property.find('span', {"class": "p24_featureDetails", 'title':'Parking Spaces'}).text.strip()
            except:
                parking_spaces = 'na'

            try:
                floor_size = property.find('span', {"class": "p24_size", 'title':'Floor Size'}).text.strip().rstrip('m²').strip()
            except:
                floor_size = 'na'

            temp = pd.DataFrame([[id, title, description, bedrooms, bathrooms, parking_spaces, floor_size, location, address, rental_term, price]], columns=columns)

            df = pd.concat([df, temp], ignore_index=True)

            del temp
    
    except Exception as ex:
        print(ex)
        print("probably last page:", page)
        break # exit `while` loop

In [ ]:
df.to_feather('western_cape_rentals.ftr')

NameError: name 'df' is not defined

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file("Western Cape Rentals.html")

Summarize dataset:  74%|███████▍  | 17/23 [00:19<00:00, 26.16it/s, scatter id, id]                 

In [ ]:
western_cape = "https://www.property24.com/for-sale/all-cities/western-cape/9"
gauteng = "https://www.property24.com/for-sale/all-cities/gauteng/1"

In [ ]:
# Query Categories
ACTIONS = ["for-sale", "to-rent"]
PROPERTY_CATEGORIES = ["House", "ApartmentOrFlat", "Townhouse", "VacantLandOrPlot", "Farm", "Commercial", "Industrial"]
PARKING_TYPES = ["ParkingOrGarage", "GarageOnly"]

# Query Syntax
QUERY = "?"
OR = "%2c"
AND = "%26"
MIN_PRICE = "pf%3d"
MAX_PRICE = "pt%3d"
BEDROOMS = "bd%3d"
BATHROOM = "bth%3d"
PARKING_SPACES = "ps%3d"
PARKING_TYPE = "pst%3d"

# Query variables
PROPERTY_CATEGORY = "PropertyCategory="
FILTER = "sp="

In [2]:
URL = "https://www.property24.com/to-rent/somerset-west-central/somerset-west/western-cape/9019/110768093?plId=967801&plt=3"

page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
results = soup.find_all("div", class_="p24_listingContent")

In [4]:
for result in results:
    price = result.find("div", class_="p24_listingCard p24_listingFeaturesWrapper")
    # company_element = result.find("h3", class_="company")
    # location_element = result.find("p", class_="location")

    print(price.text.strip())

R 8 500


In [5]:
soup


<!DOCTYPE html>

<!--[if lt IE 8]><html class=" p24_live p24_outdated"><![endif]-->
<!--[if IE 8]><html class=" p24_live ie8"><![endif]-->
<!--[if IE 9]><html class=" p24_live ie9"><![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="p24_live" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<!--<![endif]-->
<head>
<link href="/manifest.json" rel="manifest"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-regular.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-300.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-600.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/fonts/source-sans-pro-v14-latin-700.woff?60dcc3" rel="preload" type="font/woff"/>
<link as="font" crossorigin="anonymous" href="/Content/font-awesome/